# Cloud Run Management

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed PromptFlow SDK


**Learning Objectives** - By the end of this tutorial, you should be able to:
- create run with remote data
- manage runs via run.yaml
- serverless runtime


**Motivations** - This guide will walk you through cloud run management abilities.

## 0. Install promptflow sdk

In [ ]:
!pip install "promptflow[azure]" promptflow-tools --extra-index-url https://azuremlsdktestpypi.azureedge.net/promptflow/

## Create run with remote data

### 1.1 Import the required libraries

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.entities import Data
from azure.core.exceptions import ResourceNotFoundError

from promptflow.azure import PFClient
from promptflow.entities import Run

### 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

### 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

pf = PFClient(ml_client)

runtime = "demo-mir"  # TODO remove this after serverless ready

### 1.4 Create or update remote data

In [ ]:
data_name, data_version = "flow_run_test_data", "1"

try:
    data = ml_client.data.get(name=data_name, version=data_version)
except ResourceNotFoundError:
    data = Data(
        name=data_name,
        version=data_version,
        path=f"../../flows/standard/web-classification/data.jsonl",
        type="uri_file",
    )
    data = ml_client.data.create_or_update(data)

### 1.5 Prepare remote data id

In [ ]:
data_id = f"azureml:{data.name}:{data.version}"
print(data_id)

### Create a flow run with remote data

In [ ]:
# create run
run = Run(
    # local flow file
    flow="../../flows/standard/web-classification",
    # remote data
    data=data_id,
)

base_run = pf.runs.create_or_update(
    run=run,
    runtime="demo-mir",
)

### Stream the flow run to make sure it runs successfully

In [ ]:
pf.runs.stream(base_run)